In [1]:
import MySQLdb
import pandas as pd
import numpy as np
from datetime import datetime

In [116]:
hc = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_non/herb/herbal_compound_positive_name_herb_info.tsv")

In [117]:
hc.head(1)

,herbID,herbName,Herbal_compID,compName,synonym,Disease_id,OMIM_disease_name,avg
0,[3349],['Solanum aculeatissimum'],141,Pyridoxal,3-HYDROXY-5-(hydroxymethyl)-2-methylisonicotin...,239350,"Hyperphosphatemia, Polyuria, And Seizures",0.952639


In [7]:
hc_ids = hc.Herbal_compID.drop_duplicates().tolist()
print len(hc_ids)

31


In [8]:
#접속하기
db_handle = MySQLdb.connect('localhost', 'root','comsysbio', 'COCONUTv5')
#cur = db_handle.cursor(MySQLdb.cursors.DictCursor) # query를 통해 데이터를 얻을 때 python의 dict로 얻기 위한 방법
cursor = db_handle.cursor()

In [9]:
"{}".format(tuple(hc_ids))

'(141, 153, 416, 562, 614, 622, 752, 760, 845, 926, 1001, 1020, 1166, 1167, 1529, 2969, 4020, 4286, 5529, 7859, 7874, 7996, 8012, 8019, 8024, 9926, 10086, 10195, 78876, 83229, 86025)'

In [10]:
# herbal compound 관련 herb ID 뽑기
query1 = "SELECT herbID, compID FROM Herb_Compound where compID in {}".format(tuple(hc_ids))
cursor.execute(query1); #쿼리 날리는 거
results = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [26]:
query1

'SELECT herbID, compID FROM Herb_Compound where compID in (141, 153, 416, 562, 614, 622, 752, 760, 845, 926, 1001, 1020, 1166, 1167, 1529, 2969, 4020, 4286, 5529, 7859, 7874, 7996, 8012, 8019, 8024, 9926, 10086, 10195, 78876, 83229, 86025)'

In [38]:
np_results = np.array(results)
comp_herb = pd.DataFrame({'herbID': np_results[:, 0], 'compID' : np_results[:, 1]}).drop_duplicates()

In [148]:
print len(comp_herb), len(comp_herb.compID.drop_duplicates()), len(comp_herb.herbID.drop_duplicates())

752 31 376


In [27]:
hb_ids = df_results.herbID.drop_duplicates().tolist()
hb_ids = [int(x) for x in hb_ids]
print len(hb_ids)

376


In [29]:
# herbID로 herb phenID, phenotype name, umls id 뽑기
query2 = "SELECT phenID, name, umlsID FROM Phenotype WHERE phenID in (SELECT phenID FROM Herb_Phenotype where herbID in {})".format(tuple(hb_ids))
cursor.execute(query2); #쿼리 날리는 거
results2 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [41]:
np_results2 = np.array(results2)
phe_umls = pd.DataFrame({'phenID': np_results2[:, 0], 'name' : np_results2[:, 1], 'umlsID' : np_results2[:, 2]}).drop_duplicates()

In [150]:
print len(phe_umls), len(phe_umls.phenID.drop_duplicates()), len(phe_umls.umlsID.drop_duplicates())

 1110 1110 1110


In [37]:
# herbID로 herb, phen id 뽑기
query3 = "SELECT herbID, phenID FROM Herb_Phenotype where herbID in {}".format(tuple(hb_ids))
cursor.execute(query3); #쿼리 날리는 거
results3 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [63]:
np_results3 = np.array(results3)
herb_phe = pd.DataFrame({'herbID': np_results3[:, 0], 'phenID' : np_results3[:, 1]}).drop_duplicates()

In [64]:
herb_phe.head(1)

,herbID,phenID
0,3,8940


In [151]:
print len(herb_phe), len(herb_phe.herbID.drop_duplicates()), len(herb_phe.phenID.drop_duplicates())

7202 238 1110


In [152]:
len(comp_herb[comp_herb.herbID.isin(herb_phe.herbID)].compID.drop_duplicates())

29

In [156]:
len(hc[hc.Herbal_compID.isin(comp_herb[comp_herb.herbID.isin(herb_phe.herbID)].compID)].Disease_id.drop_duplicates())

85

In [45]:
# herbID로 herb name, phen id 뽑기
query4 = "SELECT herbID, name FROM Herb where herbID in {}".format(tuple(hb_ids))
cursor.execute(query4); #쿼리 날리는 거
results4 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [79]:
np_results4 = np.array(results4)
herb_name = pd.DataFrame({'herbID': np_results4[:, 0], 'name' : np_results4[:, 1]}).drop_duplicates()

In [84]:
herb_name.herbID = [int(i) for i in herb_name.herbID]

In [86]:
phe_umls.phenID = [int(i) for i in phe_umls.phenID]

In [93]:
herb_name.rename(columns={'name' : 'herbName'}, inplace=True)

In [94]:
phe_umls.rename(columns={'name' : 'phenName'}, inplace=True)

In [96]:
merged=comp_herb.merge(herb_name, on='herbID').merge(herb_phe, on='herbID').merge(phe_umls, on='phenID')

In [99]:
merged.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_non/herb/herb_compound_phenotype_umls.tsv", sep='\t', index=False)

In [102]:
merged.head(1)

,compID,herbID,herbName,phenID,phenName,umlsID
0,141,3349,Solanum aculeatissimum,840,Cough,C1961131


In [147]:
print len(merged.compID.drop_duplicates()), len(merged.herbID.drop_duplicates()),len(merged.phenID.drop_duplicates()), len(merged.umlsID.drop_duplicates())

29 238 1110 1110


In [100]:
mim_umls = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/1_omim_umls_mapping_association.tsv")

In [104]:
herb_mim = pd.merge(merged, mim_umls, left_on = 'umlsID', right_on='UMLS_concept_ID')

In [105]:
herb_mim

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name
0,141,3349,Solanum aculeatissimum,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
1,153,1012,Eleocharis dulcis,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
2,86025,1012,Eleocharis dulcis,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
3,153,527,Glebionis coronaria,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
4,86025,527,Glebionis coronaria,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
5,153,1785,Syzygium jambos,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
6,4020,1785,Syzygium jambos,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
7,86025,1785,Syzygium jambos,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
8,153,2607,root,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough
9,926,2607,root,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough


In [107]:
print len(herb_mim.compID.drop_duplicates()), len(herb_mim.herbID.drop_duplicates()), len(herb_mim.umlsID.drop_duplicates()), len(herb_mim.OMIM_ID.drop_duplicates())

29 198 58 66


In [112]:
herb_mim.head(1)

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name
0,141,3349,Solanum aculeatissimum,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough


In [120]:
hc.head(1)

,herbID,herbName,Herbal_compID,compName,synonym,Disease_id,OMIM_disease_name,avg
0,[3349],['Solanum aculeatissimum'],141,Pyridoxal,3-HYDROXY-5-(hydroxymethyl)-2-methylisonicotin...,239350,"Hyperphosphatemia, Polyuria, And Seizures",0.952639


In [118]:
renamed = hc[['Herbal_compID', 'compName', 'synonym', 'Disease_id', 'avg']].rename(columns = {'Herbal_compID' : 'compID', 'Disease_id' : 'OMIM_ID'})

In [121]:
hc_herb_mim = pd.merge(herb_mim, renamed)

In [140]:
hc_herb_mim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_non/herb/herb_compound_original_predicted.tsv", sep='\t', index=False)

In [126]:
print len(hc_herb_mim)
hc_herb_mim.head(3)

73


,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
0,562,369,Arctium lappa,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough,Atropine,"(+-)-Atropine/(+-)-Hyoscyamine/(+,-)-Tropyl tr...",0.825917
1,562,653,Lycium barbarum,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough,Atropine,"(+-)-Atropine/(+-)-Hyoscyamine/(+,-)-Tropyl tr...",0.825917
2,562,826,Hyoscyamus niger,840,Cough,C1961131,608088,"Neuropathy, Hereditary Sensory And Autonomic, ...",C1961131,Cough,Atropine,"(+-)-Atropine/(+-)-Hyoscyamine/(+,-)-Tropyl tr...",0.825917


In [125]:
print len(hc_herb_mim.compID.drop_duplicates()), len(hc_herb_mim.herbID.drop_duplicates()), len(hc_herb_mim.phenID.drop_duplicates()), len(hc_herb_mim.OMIM_ID.drop_duplicates())

13 36 11 11


In [130]:
hc_herb_mim.phenName.drop_duplicates()

0              Cough
6             Asthma
8           Pain NOS
12     Pain symptoms
32         HEAD ACHE
35             Fever
37             Edema
56      Hypertension
70    LEUKEMIAS, NOS
72          Migraine
Name: phenName, dtype: object

In [129]:
hc_herb_mim[hc_herb_mim.phenName == 'LEUKEMIAS, NOS']

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
70,7859,2274,Catharanthus roseus,1412,"LEUKEMIAS, NOS",C0023418,223350,Dohle Bodies And Leukemia,C0023418,LEUKAEMIA (leukemia),(+)-Vinblastine,Vinblastine/,0.906861
71,7859,2274,Catharanthus roseus,1412,"LEUKEMIAS, NOS",C0023418,246470,"Leukemia, Acute Myelocytic, With Polyposis Col...",C0023418,LEUKAEMIA (leukemia),(+)-Vinblastine,Vinblastine/,0.929417


In [145]:
hc_herb_mim.drop_duplicates(['herbID', 'compID', 'phenID']).sort_values(by='herbID')

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
43,1020,3,Paeonia lactiflora,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
64,153,27,Nelumbo nucifera,421,Hypertension,C0020538,161900,"Renal Failure, Progressive, With Hypertension",C0020538,Hypertension (Hypertensive disease),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
57,153,127,Camellia sinensis,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
59,153,229,Fagopyrum esculentum,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
32,10195,229,Fagopyrum esculentum,78,HEAD ACHE,C0018681,145590,"Hyperthermia, Cutaneous, With Headaches And Na...",C0018681,Headaches (Headache),Salicylate,SALICYLATE,0.862167
6,7874,237,Aster tataricus,145,Asthma,C0004096,208550,"Asthma, Nasal Polyps, And Aspirin Intolerance",C0004096,Asthma,Cortisol,Hydrocortisone/,0.894694
39,1020,314,Crocus sativus,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
21,2969,331,Oryza sativa,395,Pain symptoms,C0030193,147530,Insensitivity To Pain With Hyperplastic Myelin...,C0030193,Pain,O-Acetylcholine,Acetylcholine Chloride/Acetylcholine/,0.739000
48,2969,331,Oryza sativa,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,O-Acetylcholine,Acetylcholine Chloride/Acetylcholine/,0.700389
30,10195,336,Isatis tinctoria,444,Pain NOS,C1962977,147530,Insensitivity To Pain With Hyperplastic Myelin...,C1962977,Pain NOS (Pain NOS Adverse Event),Salicylate,SALICYLATE,0.858389


In [139]:
hc_herb_mim[hc_herb_mim.phenName == 'Hypertension'].sort_values(by='herbID')

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
64,153,27,Nelumbo nucifera,421,Hypertension,C0020538,161900,"Renal Failure, Progressive, With Hypertension",C0020538,Hypertension (Hypertensive disease),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
57,153,127,Camellia sinensis,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
59,153,229,Fagopyrum esculentum,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
62,153,657,Apium graveolens,421,Hypertension,C0020538,161900,"Renal Failure, Progressive, With Hypertension",C0020538,Hypertension (Hypertensive disease),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
58,153,681,Diospyros kaki,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
66,926,681,Diospyros kaki,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Salicylic acid,2-Carboxyphenol/2-Hydroxybenzoic acid/O-carbox...,0.607139
60,153,697,Helianthus annuus,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500
68,7859,2274,Catharanthus roseus,492,Hypertension,C1963138,161900,"Renal Failure, Progressive, With Hypertension",C1963138,Hypertension (Hypertension Adverse Event),(+)-Vinblastine,Vinblastine/,0.810250
69,7859,2274,Catharanthus roseus,421,Hypertension,C0020538,161900,"Renal Failure, Progressive, With Hypertension",C0020538,Hypertension (Hypertensive disease),(+)-Vinblastine,Vinblastine/,0.810250
63,153,2365,Zea mays,421,Hypertension,C0020538,161900,"Renal Failure, Progressive, With Hypertension",C0020538,Hypertension (Hypertensive disease),Eicosapentanoic acid,"(5Z,8Z,11Z,14Z,17Z)-5,8,11,14,17-Eicosapentaen...",0.778500


In [132]:
hc_herb_mim[hc_herb_mim.phenName == 'Asthma']

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
6,7874,237,Aster tataricus,145,Asthma,C0004096,208550,"Asthma, Nasal Polyps, And Aspirin Intolerance",C0004096,Asthma,Cortisol,Hydrocortisone/,0.894694
7,7874,237,Aster tataricus,145,Asthma,C0004096,600807,"Asthma, Susceptibility To",C0004096,Asthma,Cortisol,Hydrocortisone/,0.875917


In [142]:
hc_herb_mim[hc_herb_mim.phenName == 'Edema'].drop_duplicates(['herbID', 'compID']).sort_values(by='herbID')

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
43,1020,3,Paeonia lactiflora,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
39,1020,314,Crocus sativus,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
48,2969,331,Oryza sativa,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,O-Acetylcholine,Acetylcholine Chloride/Acetylcholine/,0.700389
40,1020,336,Isatis tinctoria,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
52,614,397,Vulpes vulpes peculiosa,553,Edema,C0013604,215140,Hydrops-Ectopic Calcification-Moth-Eaten Skele...,C0013604,Hydrops (Edema),Testosterone,17beta-hydroxy-4-androsten-3-one/17beta-Hydrox...,0.924528
50,614,498,Cervus elaphus,553,Edema,C0013604,215140,Hydrops-Ectopic Calcification-Moth-Eaten Skele...,C0013604,Hydrops (Edema),Testosterone,17beta-hydroxy-4-androsten-3-one/17beta-Hydrox...,0.924528
41,1020,792,Daemonorops draco,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,Probenecid,4-((Dipropylamino)sulfonyl)benzoic acid/4-(Di-...,0.674722
45,2969,794,Solanum melongena,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,O-Acetylcholine,Acetylcholine Chloride/Acetylcholine/,0.700389
37,10086,903,Allium sativum,553,Edema,C0013604,215140,Hydrops-Ectopic Calcification-Moth-Eaten Skele...,C0013604,Hydrops (Edema),(+)-Prostaglandin F2alpha,DINOPROST,0.812917
49,2969,1089,Lonicera japonica,553,Edema,C0013604,600351,"Enteropathy, Familial, With Villous Edema And ...",C0013604,Edema,O-Acetylcholine,Acetylcholine Chloride/Acetylcholine/,0.700389


In [136]:
hc_herb_mim[hc_herb_mim.phenName == 'Migraine']

,compID,herbID,herbName,phenID,phenName,umlsID,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,compName,synonym,avg
72,614,5097,"Moschus moschiferus, Moschus berezovskii, Mosc...",564,Migraine,C0149931,300125,"Migraine, Familial Typical, Susceptibility To, 2",C0149931,Migraine (Migraine Disorders),Testosterone,17beta-hydroxy-4-androsten-3-one/17beta-Hydrox...,0.653417
